In [1]:
import single_head
import multi_head
import copy
import numpy as np
import matplotlib.pyplot as plt
import cv2
from scipy.ndimage.morphology import binary_fill_holes
import sys
np.set_printoptions(threshold=sys.maxsize)
from scipy.signal import detrend
import matplotlib.gridspec as gridspec
%load_ext autoreload
%autoreload 2

<IPython.core.display.Javascript object>

In [2]:
# gen pickled heads
Sequence=1

In [6]:
for i in range(1,16):
    print(i)
    head = single_head.SingleHead.read_from_file(Sequence,i)
    head.full_filter()
    head.save()

1
depth filter done.
dangling removal done
color filter done.


In [7]:
# load heads
all_heads=[single_head.SingleHead.load_from_pickle(Sequence,i) for i in range (1,16)]

In [3]:
mhead=multi_head.MultiHead.load_from_pickle(1)

In [36]:
# make head from first to last
def make_mhead(all_heads, first, last):
    all_heads[first].reset_positions()
    all_heads[first].reset_colors()
    all_heads[first+1].reset_positions()
    all_heads[first+1].reset_colors()   
    heads = multi_head.MultiHead.joined_heads(all_heads[first], all_heads[first+1])
    heads.links.append(multi_head.Link(left=all_heads[first+1].frame_id,right=all_heads[first].frame_id))
    for i in range(first+2,last+1):
        heads.links.append(multi_head.Link(left=all_heads[i].frame_id,right=all_heads[i-1].frame_id))
        all_heads[i].reset_positions()
        all_heads[i].reset_colors()
        heads.append(all_heads[i])        
        if i==last:
             heads.links.append(multi_head.Link(left=all_heads[first].frame_id,right=all_heads[i].frame_id))
    print ("completed")
    return heads
mhead = make_mhead(all_heads, 0, 14)
mhead.calc_keypoints()

completed


In [39]:
idx=5
def calc_connections(mhead):
    for link in mhead.links[8:]:
        link.reset()
        mhead.ransac_from_link(link)        
calc_connections(mhead)

C:\Data Science\Advanced Vision\Coursework\AV_CW\SIFT.py:89: RuntimeWarning: invalid value encountered in sqrt
  err = np.sqrt(np.var(dist) / (np.sum(inliers) - min_num_inliers))


C:\Data Science\Advanced Vision\Coursework\AV_CW\SIFT.py:89: RuntimeWarning: divide by zero encountered in double_scalars
  err = np.sqrt(np.var(dist) / (np.sum(inliers) - min_num_inliers))


In [138]:
mhead.reset_all_head_positions()
for idx in range(14):
    mhead.transform_from_link(mhead.links[idx])
    mhead.links[idx].print_short()
center, distances=mhead.left_eye_deviation()
np.mean(distances)    

2-1, matches=111, Count=78, Err=0.0030
3-2, matches=132, Count=86, Err=0.0036
4-3, matches=91, Count=54, Err=0.0053
5-4, matches=105, Count=80, Err=0.0027
6-5, matches=94, Count=67, Err=0.0051
7-6, matches=83, Count=53, Err=0.0056
8-7, matches=62, Count=39, Err=0.0030
9-8, matches=16, Count=12, Err=0.0131
10-9, matches=8, Count=5, Err=nan
11-10, matches=22, Count=19, Err=0.0030
12-11, matches=55, Count=27, Err=0.0073
13-12, matches=66, Count=38, Err=0.0031
14-13, matches=78, Count=33, Err=0.0070
15-14, matches=73, Count=43, Err=0.0079
0 172576
coordinate of the left eye: [ 0.03719031 -0.17593989 -0.02675807] in frame 0
1 172581
coordinate of the left eye: [ 0.03680702 -0.17636296 -0.02357704] in frame 1
12 169336
coordinate of the left eye: [ 0.14236855 -0.09913899  0.07720867] in frame 12
13 170000
coordinate of the left eye: [ 0.14360099 -0.09760023  0.07927651] in frame 13
14 169380
coordinate of the left eye: [ 0.14376631 -0.10052259  0.08204838] in frame 14
[ 0.10074664 -0.1299129

0.08060435827744963

In [114]:
mhead.create_spheres(0.1 ,"name")

In [139]:
mhead.reset_all_head_positions()
for idx in range(6,14):
    mhead.transform_from_link(mhead.links[idx])
    mhead.links[idx].print_short()
for idx in range(5,-1,-1):
    mhead.transform_from_link(mhead.links[idx])
    mhead.links[idx].print_short()
center, distances=mhead.left_eye_deviation()
np.mean(distances)

8-7, matches=62, Count=39, Err=0.0030
9-8, matches=16, Count=12, Err=0.0131
10-9, matches=8, Count=5, Err=nan
11-10, matches=22, Count=19, Err=0.0030
12-11, matches=55, Count=27, Err=0.0073
13-12, matches=66, Count=38, Err=0.0031
14-13, matches=78, Count=33, Err=0.0070
15-14, matches=73, Count=43, Err=0.0079
7-6, matches=83, Count=53, Err=0.0056
6-5, matches=94, Count=67, Err=0.0051
5-4, matches=105, Count=80, Err=0.0027
4-3, matches=91, Count=54, Err=0.0053
3-2, matches=132, Count=86, Err=0.0036
2-1, matches=111, Count=78, Err=0.0030
0 172576
coordinate of the left eye: [ 0.11007199 -0.08181692  0.19095517] in frame 0
1 172581
coordinate of the left eye: [ 0.10754545 -0.08363845  0.19009292] in frame 1
12 169336
coordinate of the left eye: [-0.04264685 -0.03317539  0.23657184] in frame 12
13 170000
coordinate of the left eye: [-0.04539751 -0.03241294  0.2366952 ] in frame 13
14 169380
coordinate of the left eye: [-0.04691279 -0.03609478  0.23732675] in frame 14
[ 0.01653206 -0.0534276

0.08060435827744986

In [137]:
only_first_n=15
mhead.reset_all_head_positions()
link_index,err=mhead.get_next_unpositioned_link()
positioned_head_count=0
joined_heads=set() 
while(not link_index is None) and (positioned_head_count<only_first_n or only_first_n==-1):
    mhead.links[link_index].print_short()
    joined_heads.add(mhead.links[link_index].left)
    joined_heads.add(mhead.links[link_index].right)
    mhead.heads[mhead.links[link_index].right-1].background_color=np.asarray([0,0,1])
    mhead.heads[mhead.links[link_index].left-1].background_color=np.asarray([0,0,1]) 
    foo= mhead.transform_from_link(mhead.links[link_index])   
    link_index,err=mhead.get_next_unpositioned_link()
    positioned_head_count=max(positioned_head_count+1,2)
joined_heads=sorted(joined_heads)
print(joined_heads)
no_heads= len(joined_heads)
joined_heads=[str(h) for h in joined_heads]
name= f"Seq_{mhead.heads[0].sequence_id}_frames_{'_'.join(joined_heads)}"
mhead.create_spheres(min(2/max(no_heads,0.2),1),name=name)
center, distances=mhead.left_eye_deviation()
np.mean(distances)

any_head_positioned False
1-15, matches=124, Count=91, Err=0.0024
any_head_positioned True
2-1, matches=111, Count=78, Err=0.0030
any_head_positioned True
3-2, matches=132, Count=86, Err=0.0036
any_head_positioned True
4-3, matches=91, Count=54, Err=0.0053
any_head_positioned True
5-4, matches=105, Count=80, Err=0.0027
any_head_positioned True
6-5, matches=94, Count=67, Err=0.0051
any_head_positioned True
7-6, matches=83, Count=53, Err=0.0056
any_head_positioned True
8-7, matches=62, Count=39, Err=0.0030
any_head_positioned True
15-14, matches=73, Count=43, Err=0.0079
any_head_positioned True
14-13, matches=78, Count=33, Err=0.0070
any_head_positioned True
13-12, matches=66, Count=38, Err=0.0031
any_head_positioned True
12-11, matches=55, Count=27, Err=0.0073
any_head_positioned True
11-10, matches=22, Count=19, Err=0.0030
any_head_positioned True
9-8, matches=16, Count=12, Err=0.0131
any_head_positioned True
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
0 172576
coordinate of th

0.002386321508537089

In [51]:
mhead.create_spheres(min(2/max(no_heads,0.2),1),name=name)
mhead.save_spheres("boe")

In [117]:
center, distances=mhead.left_eye_deviation(1)
np.mean(distances)

0 153974
coordinate of the left eye: [-0.04270943 -0.22482576 -0.0096941 ] in frame 0
1 150143
coordinate of the left eye: [-0.046029   -0.23158732 -0.01141736] in frame 1
2 150124
coordinate of the left eye: [-0.07649181 -0.22327776  0.02547467] in frame 2
13 147576
coordinate of the left eye: [-0.01553875 -0.23271609 -0.01817739] in frame 13
14 150775
coordinate of the left eye: [-0.04775565 -0.22426942 -0.00866981] in frame 14
[-0.04570493 -0.22733527 -0.0044968 ]
[[ 0.0029955   0.00250951 -0.0051973 ]
 [-0.00032407 -0.00425205 -0.00692056]
 [-0.03078688  0.00405751  0.02997147]
 [ 0.03016618 -0.00538082 -0.01368059]
 [-0.00205072  0.00306585 -0.00417301]]
mean coordinate: [-0.04570493 -0.22733527 -0.0044968 ]. Distance to each points: [0.00650251 0.00812891 0.04315767 0.03355756 0.00556946].


0.019383222972255216

In [ ]:
for i, link in enumerate(mhead.links):
    print (i)
    link.print_short()

In [ ]:
    link_index,err=mhead.get_next_unpositioned_link()
    positioned_head_count=max(positioned_head_count+1,2)
joined_heads=sorted(joined_heads)
print(joined_heads)
no_heads= len(joined_heads)
joined_heads=[str(h) for h in joined_heads]
name= f"Seq_{mhead.heads[0].sequence_id}_frames_{'_'.join(joined_heads)}"
mhead.create_spheres(min(2/max(no_heads,0.2),1),name=name)

In [ ]:
foo= mhead.transform_from_link(mhead.links[link_index])

In [ ]:
mhead.reset_all_head_positions()
mhead.transform_from_link(mhead.links[0])
mhead.transform_from_link(mhead.links[1])
mhead.create_spheres(0.3)


In [123]:
import pandas as pd
df=pd.DataFrame(columns=['Right', 'Left',  'Matches', 'Inliers','Variance'])
for link in mhead.links:
    df = df.append({'Right' : int(link.right) , 'Left' : int(link.left) ,  'Variance' : link.err, 'Inliers' : sum(link.inliers), 'Matches' : len(link.matches)} , ignore_index=True)
df[['Right', 'Left',  'Matches', 'Inliers']] = df[['Right', 'Left',  'Matches', 'Inliers']].astype(int)
df=df.rename(columns={'Variance':"Transformation MSE"})
print(df.to_latex(index=False))

\begin{tabular}{rrrrr}
\toprule
 Right &  Left &  Matches &  Inliers &    \textbackslash sigma \\
\midrule
     1 &     2 &      111 &       78 &  0.002994 \\
     2 &     3 &      132 &       86 &  0.003627 \\
     3 &     4 &       91 &       54 &  0.005317 \\
     4 &     5 &      105 &       80 &  0.002748 \\
     5 &     6 &       94 &       67 &  0.005150 \\
     6 &     7 &       83 &       53 &  0.005607 \\
     7 &     8 &       62 &       39 &  0.003021 \\
     8 &     9 &       16 &       12 &  0.013064 \\
     9 &    10 &        8 &        5 &       NaN \\
    10 &    11 &       22 &       19 &  0.002995 \\
    11 &    12 &       55 &       27 &  0.007341 \\
    12 &    13 &       66 &       38 &  0.003067 \\
    13 &    14 &       78 &       33 &  0.007001 \\
    14 &    15 &       73 &       43 &  0.007859 \\
    15 &     1 &      124 &       91 &  0.002372 \\
\bottomrule
\end{tabular}



In [119]:
df

,Right,Left,Matches,Inliers,Variance
0,1,2,111,78,0.002994
1,2,3,132,86,0.003627
2,3,4,91,54,0.005317
3,4,5,105,80,0.002748
4,5,6,94,67,0.005150
5,6,7,83,53,0.005607
6,7,8,62,39,0.003021
7,8,9,16,12,0.013064
8,9,10,8,5,NaN
9,10,11,22,19,0.002995


In [ ]:
mhead.links[0].left

In [ ]:
mhead.create_mesh(f"{Sequence}_mesh")

In [ ]:
nrows,ncols=3,2
screenwidth=12
plt.figure(figsize = (screenwidth,3/4*nrows*screenwidth/ncols))
gs1 = gridspec.GridSpec(nrows, ncols)
gs1.update(wspace=0.1, hspace=0.1)
for row in range(nrows):
    for col in range(ncols):
        idx=ncols*row+col
        ax=plt.subplot(gs1[idx])
        mhead.heads[idx].background_color=np.array([0,0,1])
        img= mhead.heads[idx].get_filtered_image()
        ax.imshow(img)
        ax.set_axis_off()        
        ax.text(x=10, y=463 , s=f"S_{Sequence}:{idx+1}/15",horizontalalignment='left',
                              verticalalignment='bottom',  bbox=dict(facecolor='white', alpha=1))        
        del ax
plt.savefig(f"2d_filtered_seq_{Sequence}")
plt.show()

In [ ]:
sequence_ids=[0,1,2]
head_ids=[0,5,7]
abcd="abcd"
nrows,ncols=3,4
screenwidth=12
plt.figure(figsize = (screenwidth,3/4*nrows*screenwidth/ncols))
gs1 = gridspec.GridSpec(nrows, ncols)
gs1.update(wspace=0.1, hspace=0.1)
for row in range(nrows):
    head = sequence_ids[row]
    head = sequence_ids[row]
    sequence= sequence_ids[row]
    head_idx=head_ids[row]
    for col in range(ncols):
#         print (sequence, head_idx, col)
        head=mheads[sequence].heads[head_idx]
        if col==0:
            img= head.twoD_image            
            
        elif col==1:
            head.reset_filters()
            head.filter_depth(1)
            head.background_color=np.array([0,0,1])
            img= head.get_filtered_image()
        elif col==2:
            head.remove_dangling()
            img= head.get_filtered_image()
        elif col==3:
            head.remove_background_color()
            head.background_color=np.array([0,0,1])
            img= head.get_filtered_image()
        idx=ncols*row+col
        ax=plt.subplot(gs1[idx])
        ax.imshow(img)
        ax.set_axis_off()        
        if col==0:
            ax.text(x=20, y=20 , s=f"S_{sequence+1}:{head_idx+1}/15",horizontalalignment='left',
                                  verticalalignment='top',  bbox=dict(facecolor='white', alpha=1))        

        ax.text(x=20, y=453 , s=f"{abcd[col]}",horizontalalignment='left',
                              verticalalignment='bottom',  bbox=dict(facecolor='white', alpha=1))        
        
        del ax
        
plt.savefig(f"2d_full_seq_{Sequence}")
plt.show()

In [ ]:
linkindex=6
mhead.reset_all_head_positions()
foo= mhead.transform_from_link(mhead.links[linkindex])
mhead.links[linkindex].print_short()
mhead.create_spheres(0.4)

In [ ]:
import glob

In [ ]:
glob.glob("images\Seq_1_frames_SIFT*.png")